<a href="https://colab.research.google.com/github/skybristol/experiments/blob/dev/Extracted_PDF_Annotation_via_Zotero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I'm experimenting here with a process to turn annotations created within PDF files stored as part of a Zotero library into metadata contents and structured annotations for the bibliographic record. This is essentially for cases where there is no good citation metadata already in existence somewhere on the web (e.g., for certain types of government reports) and we need to extract that content from within PDFs. It's also for cases where built-in structured PDF metadata is no good, which is the case for anything other than professionally built PDFs (e.g., just exporting a PDF from your word processor does not build a good PDF). This technique also holds promise for setting up training data for building various kinds of entity recognition models to auto-extract particular concepts from full texts processed with NLP.

I used the ZotFile plugins for Zotero, inspired by [this video](https://www.youtube.com/watch?v=_Fjhad-Z61o&t=1251s). In Zotero, the process includes storing the PDF file as an attachment so that Zotero is "managing" it, annotating the file using some type of PDF markup tool (I used Preview on Mac), and then running the ZotFile tool to extract annotations from the PDF which creates a rich text (html) note for the item in Zotero. The notes are synced to the group library with Zotero online where they can be picked up for processing.

For annotation, I used a combination of highlighting particular text and then tagging that text with a keyword corresponding to a target part of the citation metadata I'm trying to identify (e.g., title, authors, etc.). I should then be able to pull these two pieces out of the generated markdown into a data structure that I can feed back into the corresponding record via the Zotero API.

For the Python processing workflow part of this, I used the pyzotero package to connect to the Zotero group library, read items with notes, and then processed those items to generate structured data from the HTML notes that can be reinjected back into the items.

In [4]:
!pip install pyzotero

     |████████████████████████████████| 1.2 MB 7.7 MB/s 
     |████████████████████████████████| 46 kB 3.2 MB/s 
  Created wheel for feedparser: filename=feedparser-5.2.1-py3-none-any.whl size=44954 sha256=5a9afa613d6ac7007f45042fc939fda4ad84bc28647b5f341b4383b88bc84a3d
  Stored in directory: /root/.cache/pip/wheels/29/bf/46/b4a597d435d3aee6c2fa583824897336d65abf13ebe3405b70
  Created wheel for bibtexparser: filename=bibtexparser-1.2.0-py3-none-any.whl size=36711 sha256=1b79ffad0f809d0c13bc09b14dbd35ca6b619922af6344ad3be8819cf2c478be
  Stored in directory: /root/.cache/pip/wheels/3e/13/1d/09c37a40f39ddd7b226719a797f1896a5b95d730de27e7a505
Successfully built feedparser bibtexparser


In [34]:
from bs4 import BeautifulSoup
import re
from pyzotero import zotero
from getpass import getpass

This function handles the process of working a given Zotero child item's note content to detect if it was generated using the ZotFile process and then extracting the annotations into a usable data structure (list of dictionaries).

In [77]:
meta_keywords=["title","author","date","institution","project","place","commodity"]
def structured_annotations(item_key, annotation_html, extract_keywords=meta_keywords):
    annotations_soup = BeautifulSoup(annotation_html, 'html.parser')
    pattern = '\"(.*?)\"'

    paragraphs = annotations_soup.find_all("p")

    if "Extracted Annotations" not in paragraphs[0].text:
        return None

    annotation_texts = list()
    for index,p in enumerate(paragraphs):
        if p.text.split()[0] in extract_keywords:
            prop = p.text.split()[0]
            annotation_text = re.search(pattern, paragraphs[index - 1].text)
            if annotation_text is not None:
                annotation_texts.append({
                    "item_key": item_key,
                    "text": annotation_text.group(1),
                    "property": prop
                })
    return annotation_texts

To interface with the Zotero API, you need to provide a library ID and an API key. This should work for anyone with a group library that has followed the same process I outlined above.

In [7]:
zot = zotero.Zotero(input("Library ID "), "group", getpass(prompt="API Key "))

Library ID 4373054
API Key ··········


This process would need to be worked out further in production practice, but we essentially walk the items in a given library looking for notes. There might be some more efficient way to zero in on these, but I haven't figured it out yet with pyzotero. Here, we make a pass through every item, look for items with children, get the children, and then get any that have notes. I assume that this might be additive where all note items that can be processed will be yielding annotations for the given parent item. I send the note html along with the associated item key to the function to return available structured annotations for further processing. Since every annotation will essentially contain a key/value pair (property and text content), we can simply build out an array of these with their item keys for further processing.

In [80]:
for item in zot.all_top():
    if item["meta"]["numChildren"] > 0:
        note_children = [i for i in zot.children(item["key"]) if i["data"]["itemType"] == "note"]
        if note_children:
            item_annotations = list()
            for note_child in note_children:
                extracted_annotations = structured_annotations(item["key"], note_child["data"]["note"])
                if extracted_annotations:
                    item_annotations.extend(extracted_annotations)


In this case, we got the one item where I've annotated and then extracted text snippets corresponding to specific metadata elements I'm identifying and wanting to work with.

In [81]:
item_annotations

[{'item_key': 'MFA5N9X9',
  'property': 'title',
  'text': 'NI 43-101 Technical Report Mineral Resource Estimate Bullfrog Gold Project Nye County, Nevada'},
 {'item_key': 'MFA5N9X9', 'property': 'date', 'text': 'August 9, 2017'},
 {'item_key': 'MFA5N9X9', 'property': 'author', 'text': 'Rex Bryan'},
 {'item_key': 'MFA5N9X9',
  'property': 'project',
  'text': 'Bullfrog Gold Project'},
 {'item_key': 'MFA5N9X9',
  'property': 'place',
  'text': 'Bullfrog Mining District'},
 {'item_key': 'MFA5N9X9', 'property': 'place', 'text': 'Bullfrog Hills'},
 {'item_key': 'MFA5N9X9', 'property': 'place', 'text': 'Nye County'},
 {'item_key': 'MFA5N9X9', 'property': 'place', 'text': 'Nevada'},
 {'item_key': 'MFA5N9X9', 'property': 'commodity', 'text': 'gold'},
 {'item_key': 'MFA5N9X9', 'property': 'commodity', 'text': 'silver'}]

What I did above so far is a reasonable start, but there are a few issues.

* This is pretty brittle at this point and requires a very specific convention to be followed in annotating a PDF text. This would need to be made a bit more robust in terms of dealing with text strings and different things people might do in free and open annotations. I mitigated this a little bit in the function by first looking for a set of keywords identifying the specific bits of annotation that we want to go after and then getting the highlighted text the annotation is identifying. However, some type of conventions would need to be established and followed in terms of highlighting a chunk of text and then marking up its particular significance. If we want to simply pick out the major elements of reasonably complete citation metadata, then something like I tried here should work well enough.
* I still need to work out the best way to feed everything back into building more usable report reference items in Zotero once metadata properties are extracted. That should be pretty straightforward, but I want to fiddle with the simplest workflow possible where someone would mark up a bunch of PDFs quickly, load the files to the Zotero library without making report items from them, and then see if the whole process can work from there.

My takeaway so far is that it's actually really nice and fast to simply open up a PDF file and start marking it up. Theoretically, this could be done on a whole batch of PDFs totally separate from Zotero, bulk import those to Zotero, run the ZotFile extraction on the annotations, and then generate properly documented items. For the types of files this applies to, Zotero is not going to recognize that they should be "report" type items, so that part of things would need to be handled through the API. As noted, the real point here is to train an AI to do this work, at least within some contextual boundaries. But even if it was a person sitting down doing this work, it should be much faster to open a PDF, mark it up following a particular convention to identify the important bits, and then have a system take over to parse and catalog the files.